In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from deepface import DeepFace

In [3]:
df = pd.read_csv('dataset/dataset.csv')
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,504,505,506,507,508,509,510,511,Celeb,Celeb_Label
0,0,0.102777,0.437245,0.134822,0.136755,0.125466,0.077529,0.205964,-0.155207,0.372008,...,0.173018,0.077989,0.186178,-0.378087,-0.067210,0.032544,0.509354,0.153284,Bahram_Radan,0
1,1,0.177933,0.109854,0.366296,0.102263,0.098167,0.074980,0.071582,-0.139025,0.177708,...,-0.030703,-0.030513,0.011551,-0.300933,-0.041198,0.230124,0.356374,0.103244,Bahram_Radan,0
2,2,0.062198,0.274320,0.102746,0.187975,0.236909,0.005364,0.194683,-0.192908,0.569764,...,-0.004102,-0.022593,0.283998,-0.319368,-0.075604,-0.294132,0.374689,0.201106,Bahram_Radan,0
3,3,0.088835,0.424528,0.151462,0.125330,0.001227,0.109901,0.283745,-0.213615,0.411674,...,0.146300,-0.058955,0.282109,-0.216786,-0.049504,0.195647,0.581766,0.276890,Bahram_Radan,0
4,4,0.000480,0.052611,0.010194,0.238618,0.061057,0.009371,0.291541,-0.245720,0.464195,...,-0.059220,-0.213212,0.100678,-0.270852,0.099874,-0.123306,0.273968,0.273129,Bahram_Radan,0


In [32]:
celebs = df[['Celeb']].drop_duplicates().reset_index(drop=True)
celebs

,Celeb
0,Bahram_Radan
1,Ehsan_Alikhani
2,Behnam_Bani
3,Asghar_Farhadi
4,Ebi
5,Bahare_Rahnama
6,Ali_Khamenei
7,Ali_Daei
8,Dariush_Arjmand
9,Adel_FerdowsiPour


In [4]:
X = df.drop(['Unnamed: 0','Celeb','Celeb_Label'],axis=1).values
Y = df['Celeb_Label']
Y = np.eye(30)[Y]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((1380, 512), (346, 512), (1380, 30), (346, 30))

In [5]:
model = tf.keras.models.Sequential([
tf.keras.layers.Dense(512, activation='sigmoid'),
tf.keras.layers.Dense(1080, activation='relu'),
tf.keras.layers.Dense(720, activation='relu'),
tf.keras.layers.Dense(30, activation='softmax')])

In [6]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [7]:
model.fit(X_train, Y_train, epochs=50)

Epoch 1/50
44/44 [==============================] - 3s 34ms/step - loss: 0.1772 - accuracy: 0.0717
Epoch 2/50
44/44 [==============================] - 1s 34ms/step - loss: 0.1337 - accuracy: 0.1877
Epoch 3/50
44/44 [==============================] - 1s 33ms/step - loss: 0.1027 - accuracy: 0.4971
Epoch 4/50
44/44 [==============================] - 1s 32ms/step - loss: 0.0646 - accuracy: 0.7362
Epoch 5/50
44/44 [==============================] - 1s 32ms/step - loss: 0.0410 - accuracy: 0.8522
Epoch 6/50
44/44 [==============================] - 1s 32ms/step - loss: 0.0320 - accuracy: 0.8717
Epoch 7/50
44/44 [==============================] - 1s 32ms/step - loss: 0.0242 - accuracy: 0.9014
Epoch 8/50
44/44 [==============================] - 1s 33ms/step - loss: 0.0204 - accuracy: 0.9072
Epoch 9/50
44/44 [==============================] - 1s 33ms/step - loss: 0.0186 - accuracy: 0.9174
Epoch 10/50
44/44 [==============================] - 2s 38ms/step - loss: 0.0167 - accuracy: 0.9130
Epoch 11/

In [8]:
model.evaluate(X_test, Y_test)

11/11 [==============================] - 0s 7ms/step - loss: 0.0321 - accuracy: 0.8728


[0.032120801508426666, 0.8728323578834534]

In [44]:
embedding_objs = DeepFace.represent(img_path = 'pics/Adel_FerdowsiPour.jpg', model_name='ArcFace', enforce_detection=False)
predict = model.predict(np.array([embedding_objs[0]['embedding']]))
celebs['Celeb'][np.argmax(predict)]

1/1 [==============================] - 0s 30ms/step


'Adel_FerdowsiPour'

In [10]:
model.save('celebrity_recognizer_model.h5')